In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sklearn as sk
from sklearn.model_selection import train_test_split
from PIL import Image, ImageChops
import os
import pandas as pd
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

### Pre-process

In [41]:
# resize images 
for name in os.listdir(os.path.join('./heatmap_ML3/foot/')):
    #print(name)
    foot_img = Image.open(os.path.join('./heatmap_ML3/foot/', name))
    #foot_img_gray = foot_img.convert('L')
    shell_img = Image.open(os.path.join('./heatmap_ML3/shell/', name))
    #shell_img_gray = shell_img.convert('L')

    resize_foot = foot_img.resize((128,128))
    resize_foot.save(os.path.join('./ML_heatmaps/foot_color/', name))
    resize_shell = shell_img.resize((128,128))
    resize_shell.save(os.path.join('./ML_heatmaps/shell_color/', name))

In [19]:
# flip all images as left
for name in os.listdir('./ML_heatmaps/foot/'):
    foot = Image.open('./ML_heatmaps/foot/' + name)
    shell = Image.open('./ML_heatmaps/shell/' + name)

    r_or_l = name[-5]
    if r_or_l == 'R':
        foot = ImageOps.mirror(foot)
        shell = ImageOps.mirror(shell)
        
    foot.save('./ML_heatmaps/foot_L/'+name, quality=95)
    shell.save('./ML_heatmaps/shell_L/'+name, quality=95)

### Image to 1-D vector

In [48]:
def img_to_1d(img):
    """ Returns image as 1d vector"""
    arr = np.array(img)
    #print(arr.shape)
    # make a 1-dimensional view of arr
    flat_arr = arr.ravel()
    # convert it to a matrix
    #vector = np.matrix(flat_arr)

    # # do something to the vector
    # vector[:,::10] = 128

    # # reform a numpy array of the original shape
    # arr2 = np.asarray(vector).reshape(shape)

    # # make a PIL image
    # img2 = Image.fromarray(arr2, 'RGBA')
    # img2.show()
    return flat_arr

In [ ]:
foot_vectors = []
shell_vectors = []
for name in os.listdir(os.path.join('./ML_heatmaps/foot/')):
    foot_img = Image.open(os.path.join('./ML_heatmaps/foot/', name))
    shell_img = Image.open(os.path.join('./ML_heatmaps/shell/', name))

    foot_arr = img_to_1d(foot_img)
    shell_arr = img_to_1d(shell_img)

    foot_vectors.append(foot_arr)
    shell_vectors.append(shell_arr)

In [ ]:
foot_vectors = np.asarray(foot_vectors)
shell_vectors = np.asarray(shell_vectors)
print(foot_vectors.shape)
print(shell_vectors.shape) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(foot_vectors, shell_vectors, test_size=0.20, random_state=42)

In [ ]:
len(X_test[0])

In [ ]:
plt.plot(X_train[0], y_train[0], ".", markersize=5)
plt.show()

In [ ]:
plt.plot(X_train[1], y_train[1], ".", markersize=5)
plt.show()

In [ ]:
# reg = LinearRegression().fit(X_train, y_train)
# reg.score(X_test, y_test)
# # Make predictions using the testing set
# y_pred = reg.predict(X_test)

# # The coefficients
# print("Coefficients: \n", reg.coef_)
# # The mean squared error
# print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# # The coefficient of determination: 1 is perfect prediction
# print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

In [ ]:
pipe = make_pipeline(StandardScaler(), Ridge())
scores = cross_validate(pipe, X_train, y_train, return_train_score=True)
pd.DataFrame(scores)

In [ ]:
scores_dict = {
    "alpha": 10.0 ** np.arange(-2, 6, 1),
    "mean_train_scores": list(),
    "mean_cv_scores": list(),
}
for alpha in scores_dict["alpha"]:
    pipe_ridge = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
    scores = cross_validate(pipe_ridge, X_train, y_train, return_train_score=True)
    scores_dict["mean_train_scores"].append(scores["train_score"].mean())
    scores_dict["mean_cv_scores"].append(scores["test_score"].mean())

results_df = pd.DataFrame(scores_dict)

In [ ]:
results_df

In [ ]:
pipe_ridge = make_pipeline(StandardScaler(), Ridge(alpha=100000))
pipe_ridge.fit(X_train, y_train)

In [ ]:
print(X_test[0].shape)

In [ ]:
y_hat0 = pipe_ridge.predict([X_test[0]])

In [ ]:
print(len(y_hat0[0]))

In [44]:
def oned_to_img(flat_arr, shape):
    # convert it to a matrix
    vector = np.matrix(flat_arr)

    # reform a numpy array of the original shape
    arr = np.asarray(vector).reshape(shape)

    # make a PIL image
    return arr
    

In [50]:
img = Image.open('./ML_heatmaps/foot/237136L.jpg')
arr_test = img_to_1d(img)
print(arr_test.shape)
img_test = oned_to_img(arr_test, (128,128,3))
print(img_test.shape)
img_test = Image.fromarray(img_test, 'RGB')
#rgb_im = img_test.convert('RGB')
#img_test.save('./pred_to_img/control.jpeg')

(49152,)
(128, 128, 3)


In [ ]:
#print(y_hat0[0])
arr = oned_to_img(y_hat0[0], (128,128))
img = Image.fromarray(arr, 'RGB')
#rgb_im = img.convert('RGB')
img.save(os.path.join('./pred_to_img/new_gray.jpeg'))

### Image to XY + color value

In [31]:
def get_img_coords(img):
    arr = np.asarray(img)
    xy_coords = np.flip(np.column_stack(np.where(arr>=0)), axis=1)
    #print(xy_coords)
    return xy_coords

In [32]:
def get_pixels(img, coords):
    pixels = [img.getpixel((coords[i][0], coords[i][1])) for i in range(len(coords))]
    return np.asarray(pixels)

In [5]:
def rgb_to_hex(r, g, b):
    """ returns rgb to non-prefixed hex in integer format"""
    return int(('{:X}{:X}{:X}').format(r, g, b), 16) # base value 16

print(rgb_to_hex(255, 165, 1))

1047121


In [36]:
def int_to_hex(num):
    """ returns integer as hex value without prefix"""
    h = hex(num)
    #print(h)
    return str(h[2:])
test_hex = int_to_hex(1047121)
print(test_hex)

ffa51


In [39]:
def hex_to_rgb(hex_val):
  rgb = []
  for i in (0, 2, 4):
    length = len(hex_val)
    while length != 6:
      hex_val = "0" + hex_val
      length = len(hex_val)

    decimal = int(hex_val[i:i+2], 16) # little endian
    rgb.append(decimal)
  
  return tuple(rgb)

# print(hex_to_rgb('FFA501'))
# print(hex_to_rgb(test_hex))
print(hex_to_rgb('e79b'))
print(hex_to_rgb('0e79b'))
print(hex_to_rgb('2b'))

(0, 231, 155)
(0, 231, 155)
(0, 0, 43)


In [33]:
import sys
print(sys.byteorder)

little


In [ ]:
# Test
foot_img = Image.open('./ML_heatmaps/foot/237136L.jpg')
img_gray = foot_img.convert('L')
coords = get_img_coords(img_gray)
print(coords.shape)
# print((coords[0][0], coords[0][1]))
pix = foot_img.getpixel((coords[0][0], coords[0][1]))
print(pix)
pixels = get_pixels(foot_img, coords)
print(pixels.shape)
# print(np.asarray(pixels))
hex_values = [rgb_to_hex(pixels[i][0], pixels[i][1], pixels[i][2]) for i in range(len(pixels))]
#print(hex_values)
xyhex = np.column_stack((coords,hex_values)).tolist()
print(len(xyhex))
copy = xyhex
xyhex.extend(copy)
print(len(xyhex))


In [8]:
# for color images
all_foot_xyhex = []
all_shell_hex = []
for name in os.listdir(os.path.join('./ML_heatmaps/foot/')):
    foot_img = Image.open(os.path.join('./ML_heatmaps/foot/', name))
    shell_img = Image.open(os.path.join('./ML_heatmaps/shell/', name))

    foot_gray = foot_img.convert('L')
    coords = get_img_coords(foot_gray)
    pixels = get_pixels(foot_img, coords)
    hex_values = [rgb_to_hex(pixels[i][0], pixels[i][1], pixels[i][2]) for i in range(len(pixels))]
    coords_hex = np.column_stack((coords,hex_values)).tolist()
    all_foot_xyhex.extend(coords_hex)

    shell_gray = shell_img.convert('L')
    shell_coords = get_img_coords(shell_gray)
    shell_pixels = get_pixels(shell_img, shell_coords)
    shell_hex = [rgb_to_hex(shell_pixels[i][0], shell_pixels[i][1], shell_pixels[i][2]) for i in range(len(shell_pixels))]
    all_shell_hex.extend(shell_hex)


    # foot_vectors.append(foot_xyhex)
    # shell_vectors.append(shell_hex_values)



In [9]:
foot_inputs = np.asarray(all_foot_xyhex)
shell_inputs = np.asarray(all_shell_hex)
print(foot_inputs.shape)
print(shell_inputs.shape)
# print(foot_inputs)

(18907136, 3)
(18907136,)


In [10]:
import random

In [11]:
len(foot_inputs)*0.80

15125708.8

In [12]:
# Random train/test split for ALL
num = int(len(all_foot_xyhex)*0.80)
train_idx = random.sample(range(0, len(all_foot_xyhex)), len(all_foot_xyhex)-num)
test_idx = list(set(range(0, len(all_shell_hex))) - set(train_idx))

X_train = np.array([all_foot_xyhex[i] for i in train_idx])
y_train = np.array([all_shell_hex[i] for i in train_idx])
X_test = np.array([all_foot_xyhex[i] for i in test_idx])
y_test = np.array([all_shell_hex[i] for i in test_idx])

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score,  explained_variance_score, mean_absolute_error

In [ ]:
linR = LinearRegression()
linR.fit(X_train, y_train)
y_hat = linR.predict(X_test)

In [ ]:
r_sq = linR.score(X_test, y_test) # best score is 1, could also be negative , # value of 1 indicated that regression predictions perfectly fit the data
print("r\u00b2 =",  r_sq)

mse = mean_squared_error(y_test, y_hat) # best value is zero
print("MSE =", mse)

evs = explained_variance_score(y_test, y_hat, multioutput='uniform_average')
print("Explained variance score = ", evs)

mae = mean_absolute_error(y_test, y_hat) # best value is 0
print("MAE = ", mae)

print ("root_mean_squared_error : ",np.sqrt(mse)) # greater than 0.50 means it is a bad linR

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_hat, y_test, edgecolors=(0, 0, 1))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
plt.show()

In [ ]:
residuals = y_test - y_hat
plt.scatter(y_hat, residuals)
plt.show()

In [15]:
from sklearn.pipeline import Pipeline, make_pipeline


In [18]:
pipe = make_pipeline(StandardScaler(), Ridge())
scores = cross_validate(pipe, X_train, y_train, return_train_score=True)
pd.DataFrame(scores)

,fit_time,score_time,test_score,train_score
0,0.744601,0.048780,0.591840,0.591889
1,0.622224,0.028267,0.593467,0.591482
2,0.555851,0.029780,0.590461,0.592233
3,0.630407,0.034206,0.589485,0.592474
4,0.566591,0.032764,0.594108,0.591318


In [19]:
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.5911087581075478

In [20]:
# Test
foot_img = Image.open('./ML_heatmaps/foot/237136L.jpg')
img_gray = foot_img.convert('L')
coords = get_img_coords(img_gray)
print(coords.shape)
pixels = get_pixels(foot_img, coords)
print(pixels.shape)
# print(np.asarray(pixels))
hex_values = [rgb_to_hex(pixels[i][0], pixels[i][1], pixels[i][2]) for i in range(len(pixels))]
print(hex_values)
xyhex = np.column_stack((coords,hex_values)).tolist()
print(len(xyhex))
print(type(hex_values[0]))

(16384, 2)
(16384, 3)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
y_hat = pipe.predict(xyhex)

In [22]:
y_hat_new = np.asarray([abs(int(y_hat[i])) for i in range(len(y_hat))])

In [23]:
print(y_hat_new)
hex_val = np.asarray([int_to_hex(y_hat_new[i]) for i in range(len(y_hat_new))])
print(hex_val)

[ 950373  950441  950509 ... 1124844 1124776 1124709]
0xe8065
0xe80a9
0xe80ed
0xe8130
0xe8174
0xe81b8
0xe81fc
0xe8240
0xe8284
0xe82c7
0xe830b
0xe834f
0xe8393
0xe83d7
0xe841a
0xe845e
0xe84a2
0xe84e6
0xe852a
0xe856e
0xe85b1
0xe85f5
0xe8639
0xe867d
0xe86c1
0xe8704
0xe8748
0xe878c
0xe87d0
0xe8814
0xe8858
0xe889b
0xe88df
0xe8923
0xe8967
0xe89ab
0xe89ee
0xe8a32
0xe8a76
0xe8aba
0xe8afe
0xe8b42
0xe8b85
0xe8bc9
0xe8c0d
0xe8c51
0xe8c95
0xe8cd9
0xe8d1c
0xe8d60
0xe8da4
0xe8de8
0xe8e2c
0xe8e6f
0xe8eb3
0xe8ef7
0xe8f3b
0xe8f7f
0xe8fc3
0xe9006
0xe904a
0xe908e
0xe90d2
0xe9116
0xe9159
0xe919d
0xe91e1
0xe9225
0xe9269
0xe92ad
0xe92f0
0xe9334
0xe9378
0xe93bc
0xe9400
0xe9443
0xe9487
0xe94cb
0xe950f
0xe9553
0xe9597
0xe95da
0xe961e
0xe9662
0xe96a6
0xe96ea
0xe972e
0xe9771
0xe97b5
0xe97f9
0xe983d
0xe9881
0xe98c4
0xe9908
0xe994c
0xe9990
0xe99d4
0xe9a18
0xe9a5b
0xe9a9f
0xe9ae3
0xe9b27
0xe9b6b
0xe9bae
0xe9bf2
0xe9c36
0xe9c7a
0xe9cbe
0xe9d02
0xe9d45
0xe9d89
0xe9dcd
0xe9e11
0xe9e55
0xe9e98
0xe9edc
0xe9f20
0xe9f64
0x

In [58]:
rgb = np.asarray([hex_to_rgb(hex_val[i]) for i in range(len(y_hat))])
img_arr = rgb.reshape(128, 128, 3)
img = Image.fromarray(img_arr, 'RGB')
img.save('./pred_to_img/predV2.jpeg')


In [62]:
inp = np.asarray(xyhex)
print(inp.shape)
img_arr2 = inp.reshape(128, 128, 3)
img2 = Image.fromarray(img_arr2, 'RGB')
img2.save('./pred_to_img/controlV2.jpeg')

(16384, 3)


### Image column format

In [20]:
def img_to_col(img):
    """ Returns image as 1d vector"""
    arr = np.array(img)
    flat_arr = arr.ravel()
    return flat_arr

In [22]:
def get_img_coords(img):
    arr = np.asarray(img)
    xy_coords = np.flip(np.column_stack(np.where(arr>=0)), axis=1)
    #print(xy_coords)
    return xy_coords.tolist()

In [23]:
def get_pixels(img, coords):
    pixels = [img.getpixel((coords[i][0], coords[i][1])) for i in range(len(coords))]
    return pixels

In [94]:
foot_img = Image.open('./ML_heatmaps/foot/237136L.jpg')
img_gray = foot_img.convert('L')
coords = get_img_coords(img_gray)
# print((coords[0][0], coords[0][1]))
pix = foot_img.getpixel((coords[0][0], coords[0][1]))
pxs = [foot_img.getpixel((coords[i][0], coords[i][1])) for i in range(len(coords))]
foot_input = np.column_stack((coords, pxs)).tolist()
print(len(foot_input))
foot_input.extend(foot_input)
print(len(foot_input))
print(len(foot_input[0]))



16384
32768
3


In [56]:
inputs = []
outputs = []
for name in os.listdir(os.path.join('./ML_heatmaps/foot_mod_L/')):
    foot_img = Image.open(os.path.join('./ML_heatmaps/foot_mod_L/', name))
    shell_img = Image.open(os.path.join('./ML_heatmaps/shell_L/', name))
    foot_gray = foot_img.convert('L')
    foot_coords = get_img_coords(foot_gray)
    foot_pixels = [foot_img.getpixel((foot_coords[i][0], foot_coords[i][1])) for i in range(len(foot_coords))]
    shell_coords = get_img_coords(shell_img)
    shell_pixels = [shell_img.getpixel((shell_coords[i][0], shell_coords[i][1])) for i in range(len(shell_coords))]

    foot_xyp = np.column_stack((foot_coords, foot_pixels)).tolist()

    #print(foot_arr)
    inputs.extend(foot_xyp)
    outputs.extend(shell_pixels)

foots_xyp = np.asarray(inputs)
shells_p = np.asarray(outputs)

In [57]:
print(foots_xyp.shape)
print(shells_p.shape)

(18907136, 3)
(18907136,)


In [58]:
x_train, x_test, y_train, y_test = train_test_split(foots_xyp, shells_p, test_size=0.20, random_state=42)

KeyboardInterrupt: 

In [ ]:
scores_dict = {
    "alpha": 10.0 ** np.arange(-2, 6, 1),
    "mean_train_scores": list(),
    "mean_cv_scores": list(),
}
for alpha in scores_dict["alpha"]:
    pipe_ridge = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
    scores = cross_validate(pipe_ridge, x_train, y_train, return_train_score=True)
    scores_dict["mean_train_scores"].append(scores["train_score"].mean())
    scores_dict["mean_cv_scores"].append(scores["test_score"].mean())

results_df = pd.DataFrame(scores_dict)
results_df

,alpha,mean_train_scores,mean_cv_scores
0,0.01,0.738664,0.738664
1,0.10,0.738664,0.738664
2,1.00,0.738664,0.738664
3,10.00,0.738664,0.738664
4,100.00,0.738664,0.738664
5,1000.00,0.738664,0.738664
6,10000.00,0.738664,0.738663
7,100000.00,0.738615,0.738615


In [ ]:
pipe_ridge = make_pipeline(StandardScaler(), Ridge(alpha=10))
pipe_ridge.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=10))])

In [ ]:
# predict image
for img in os.listdir('./ML_heatmaps/foot_mod_L/'):
    print(img)
    foot = Image.open('./ML_heatmaps/foot_mod_L/' + img)
    coords = get_img_coords(foot)
    pixs_img = [foot.getpixel((coords[i][0], coords[i][1])) for i in range(len(coords))]
    pred_xyp = np.column_stack((foot_coords, foot_pixels)).tolist()

    pred_pixs = pipe_ridge.predict(pred_xyp)


    pred_arr = np.array(pred_pixs, dtype=np.uint8).reshape((128,128))

    pred_img = Image.fromarray(pred_arr)
    pred_img.save('./pred_to_img/predV2_L/' + img)
    del img, foot, coords, pixs_img, pred_xyp, pred_pixs, pred_arr, pred_img

240159R.jpg
238184L.jpg
238684R.jpg
238112R.jpg
239146L.jpg
239353R.jpg
239187R.jpg
237526R.jpg
237875R.jpg
239404L.jpg
238700L.jpg
238846R.jpg
239111L.jpg
238515R.jpg
240118R.jpg
239384L.jpg
237475R.jpg
237926R.jpg
237863R.jpg
238080L.jpg
238295R.jpg
237530R.jpg
237619R.jpg
238016R.jpg
238446R.jpg
238716L.jpg
237971R.jpg
239596R.jpg
237908L.jpg
240136L.jpg
239715R.jpg
238104R.jpg
239717L.jpg
238106L.jpg
238556L.jpg
239447R.jpg
240134R.jpg
239294R.jpg
237565L.jpg
238690L.jpg
237320R.jpg
239081L.jpg
238893R.jpg
238501L.jpg
239105R.jpg
238344R.jpg
238444L.jpg
239410R.jpg
237532L.jpg
237162L.jpg
237498L.jpg
238278L.jpg
237477L.jpg
239469L.jpg
237924L.jpg
239543R.jpg
238147L.jpg
238452L.jpg
239685R.jpg
238281L.jpg
237524L.jpg
237731R.jpg
237674R.jpg
238978R.jpg
238540L.jpg
240067R.jpg
240088R.jpg
239001R.jpg
239451R.jpg
238539R.jpg
238686L.jpg
239078L.jpg
239582L.jpg
237789R.jpg
237654R.jpg
240069L.jpg
237504L.jpg
238472L.jpg
239633L.jpg
239776L.jpg
238537L.jpg
237945L.jpg
237746R.jpg
2393

In [15]:
from PIL import ImageOps

### Image rgb

In [34]:
def get_img_coords(img):
    arr = np.asarray(img)
    xy_coords = np.flip(np.column_stack(np.where(arr>=0)), axis=1)
    #print(xy_coords)
    return xy_coords

In [59]:
# for color images
all_foot_xyrgb = []
all_shell_rgb = []
for name in os.listdir(os.path.join('./ML_heatmaps/foot_color/')):
    foot_img = Image.open(os.path.join('./ML_heatmaps/foot_color/', name))
    shell_img = Image.open(os.path.join('./ML_heatmaps/shell_color/', name))

    foot_gray = foot_img.convert('L')
    coords = get_img_coords(foot_gray)

    foot_pixels = [foot_img.getpixel((foot_coords[i][0], foot_coords[i][1])) for i in range(len(foot_coords))]

    coords_rgb = np.column_stack((coords,foot_pixels)).tolist()
    all_foot_xyrgb.extend(coords_rgb)


    shell_pixels = [shell_img.getpixel((coords[i][0], coords[i][1])) for i in range(len(coords))]
    all_shell_rgb.extend(shell_pixels)
    
xyrgb = np.array(all_foot_xyrgb)
rgb = np.array(all_shell_rgb)
print(xyrgb.shape)
print(rgb.shape)

(18907136, 5)
(18907136, 3)


In [60]:
x_train, x_test, y_train, y_test = train_test_split(xyrgb, rgb, test_size=0.20, random_state=42)

In [52]:
scores_dict = {
    "alpha": 10.0 ** np.arange(-2, 6, 1),
    "mean_train_scores": list(),
    "mean_cv_scores": list(),
}
for alpha in scores_dict["alpha"]:
    pipe_ridge = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
    scores = cross_validate(pipe_ridge, x_train, y_train, return_train_score=True)
    scores_dict["mean_train_scores"].append(scores["train_score"].mean())
    scores_dict["mean_cv_scores"].append(scores["test_score"].mean())

results_df = pd.DataFrame(scores_dict)
results_df

,alpha,mean_train_scores,mean_cv_scores
0,0.01,0.616781,0.616779
1,0.10,0.616781,0.616779
2,1.00,0.616781,0.616779
3,10.00,0.616781,0.616779
4,100.00,0.616780,0.616779
5,1000.00,0.616746,0.616744
6,10000.00,0.615424,0.615423
7,100000.00,0.610329,0.610328


In [61]:
pipe_ridge = make_pipeline(StandardScaler(), Ridge(alpha=10))
pipe_ridge.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=10))])

In [65]:
scores = cross_validate(pipe_ridge, x_train, y_train, return_train_score=True)
pd.DataFrame(scores)

,fit_time,score_time,test_score,train_score
0,6.728441,0.568989,0.616369,0.616883
1,4.895518,0.444782,0.615917,0.616996
2,4.923370,0.426747,0.617920,0.616495
3,4.858338,0.427620,0.616841,0.616765
4,4.861185,0.423172,0.616850,0.616763


In [63]:
# predict image
for img in os.listdir('./ML_heatmaps/foot_color/'):
    print(img)
    foot = Image.open('./ML_heatmaps/foot_color/' + img)
    foot_gray = foot_img.convert('L')
    coords = get_img_coords(foot_gray)
    pixs_img = [foot.getpixel((coords[i][0], coords[i][1])) for i in range(len(coords))]
    pred_xyp = np.column_stack((foot_coords, foot_pixels)).tolist()

    pred_pixs = pipe_ridge.predict(pred_xyp)


    pred_arr = np.array(pred_pixs, dtype=np.uint8).reshape((128,128,3))

    pred_img = Image.fromarray(pred_arr, 'RGB')
    pred_img.save('./pred_to_img/predV3/' + img)
    del img, foot, coords, pixs_img, pred_xyp, pred_pixs, pred_arr, pred_img

240159R.jpg
238184L.jpg
238684R.jpg
238112R.jpg
239146L.jpg
239353R.jpg
239187R.jpg
237526R.jpg
237875R.jpg
239404L.jpg
238700L.jpg
238846R.jpg
239111L.jpg
238515R.jpg
240118R.jpg
239384L.jpg
237475R.jpg
237926R.jpg
237863R.jpg
238080L.jpg
238295R.jpg
237530R.jpg
237619R.jpg
238016R.jpg
238446R.jpg
238716L.jpg
237971R.jpg
239596R.jpg
237908L.jpg
240136L.jpg
239715R.jpg
238104R.jpg
239717L.jpg
238106L.jpg
238556L.jpg
239447R.jpg
240134R.jpg
239294R.jpg
237565L.jpg
238690L.jpg
237320R.jpg
239081L.jpg
238893R.jpg
238501L.jpg
239105R.jpg
238344R.jpg
238444L.jpg
239410R.jpg
237532L.jpg
237162L.jpg
237498L.jpg
238278L.jpg
237477L.jpg
239469L.jpg
237924L.jpg
239543R.jpg
238147L.jpg
238452L.jpg
239685R.jpg
238281L.jpg
237524L.jpg
237731R.jpg
237674R.jpg
238978R.jpg
238540L.jpg
240067R.jpg
240088R.jpg
239001R.jpg
239451R.jpg
238539R.jpg
238686L.jpg
239078L.jpg
239582L.jpg
237789R.jpg
237654R.jpg
240069L.jpg
237504L.jpg
238472L.jpg
239633L.jpg
239776L.jpg
238537L.jpg
237945L.jpg
237746R.jpg
2393

KeyboardInterrupt: 

### Comparison

In [1]:
def compare_images(img1, img2):
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    return m_norm

In [4]:
def abs_diff(img1, img2):
    img3 = ImageChops.difference(img1, img2)
    return img3

In [24]:
# control
test = Image.open('./ML_heatmaps/foot/237136L.jpg')
image_sequence = test.getdata()
arr = np.array(image_sequence)

diff = abs(arr-arr)
boxplot(diff, 'control')

In [6]:
import statistics as st

In [5]:
def boxplot(diff, name):
    mean = st.mean(diff)
    median = st.median(diff)
    sd = np.std(diff)

    t_mean = 'Mean = ' + str(round(mean,4))
    t_median = 'Median = ' + str(round(median,4))
    t_sd = 'SD = ' + str(round(sd,4))

    fig, ax = plt.subplots(facecolor=(1, 1, 1), figsize=(16,16))
    ax.set_title('Boxplot of Pixel Difference ' + name)
    ax.set_ylim([0,250])
    ax.set_ylabel('Absolute Difference')
    ax.text(1.2, mean, t_mean + '\n'+ t_median + '\n'+ t_sd, style='italic',bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10}, fontsize=18)

    ax.get_xticklabels() + ax.get_yticklabels()
    ax.boxplot(diff)

    plt.savefig('./comparison/all/' + name, transparent=False)
    plt.clf()
    plt.close()

In [10]:
def box_plot(foot_truth, foot_pred, pred_truth, name):
        np.random.seed(1234)
        # Difference between foot and ground-truth shell
     
        mean = st.mean(foot_truth)
        median = st.median(foot_truth)
        sd = np.std(foot_truth)

        t_mean = 'Mean = ' + str(round(mean,4))
        t_median = 'Median = ' + str(round(median,4))
        t_sd = 'SD = ' + str(round(sd,4))

        fig, ax = plt.subplots(facecolor=(1, 1, 1), figsize=(16,16))
        ax.set_title('Boxplot of Pixel Difference for ' + name)
        ax.set_ylim([0,250])
        ax.set_xlabel(' Foot vs. Ground-truth Shell          Foot vs. Predicted Shell        Ground-truth Shell vs. Predicted Shell ')
        ax.set_ylabel('Absolute Difference')
        ax.text(1.2, mean, t_mean + '\n'+ t_median + '\n'+ t_sd, style='italic',bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10}, fontsize=18)
        
        ax.boxplot([foot_truth, foot_pred, pred_truth])

        # Difference between foot and predictted shell
        mean2 = st.mean(foot_pred)
        median2 = st.median(foot_pred)
        sd2 = np.std(foot_pred)

        t_mean2 = 'Mean = ' + str(round(mean2,4))
        t_median2 = 'Median = ' + str(round(median2,4))
        t_sd2 = 'SD = ' + str(round(sd2,4))

        ax.text(2.15, mean, t_mean2 + '\n'+ t_median2 + '\n'+ t_sd2, style='italic', bbox={'facecolor': 'blue', 'alpha': 0.5, 'pad': 10}, fontsize=18)

        # Difference between predictted and ground-truth shell
        mean3 = st.mean(pred_truth)
        median3 = st.median(pred_truth)
        sd3 = np.std(pred_truth)

        t_mean3 = 'Mean = ' + str(round(mean3,4))
        t_median3 = 'Median = ' + str(round(median3,4))
        t_sd3 = 'SD = ' + str(round(sd3,4))

        ax.text(3.15, mean, t_mean3 + '\n'+ t_median3 + '\n'+ t_sd3, style='italic', bbox={'facecolor': 'green', 'alpha': 0.5, 'pad': 10}, fontsize=18)

        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
             item.set_fontsize(18)

        plt.savefig('./comparison/all/' + name[:-4], transparent=False)
        plt.clf()
        plt.close()

In [9]:
def get_non_black(shell_arr):
    """ returns coordinates of shell_img where pixels are greater than 0 (not black) """
    coords = np.column_stack(np.where(shell_arr > 10))
    return coords

In [94]:
shell = Image.open('./ML_heatmaps/shell/237136L.jpg')
pred =  Image.open('./pred_to_img/pred/237136L.jpg')
shell_arr = np.array(shell)
nonblack_coords = get_non_black(shell_arr)
print(len(nonblack_coords))

# for x,y in nonblack_coords:
#     shell.putpixel( (y,x), (255) )
# pred_arr = np.array(shell, dtype=np.uint8).reshape((128,128))

# pred_img = Image.fromarray(pred_arr)
# pred_img.save('./pred_to_img/change4.png')

pred_shell = [pred.getpixel((y,x)) for x,y in nonblack_coords]
true_shell = [shell.getpixel((y,x)) for x,y in nonblack_coords]
pred_shell = np.array(pred_shell)
true_shell = np.array(true_shell)
diff = abs(pred_shell - true_shell)
boxplot(diff, '/aligned/test')


1851


In [6]:
import cv2
import numpy as np
import statistics as st

In [11]:
# HERE
for name in ['240136L.jpg', '238501L.jpg', '238184L.jpg']:
    # shell and predicted
    shell = Image.open('./ML_heatmaps/shell/' + name)
    pred =  Image.open('./pred_to_img/pred/' + name)
    shell_arr = np.array(shell)
    nonblack_coords = get_non_black(shell_arr)
    pred_shell = [pred.getpixel((y,x)) for x,y in nonblack_coords]
    true_shell = [shell.getpixel((y,x)) for x,y in nonblack_coords]
    pred_shell = np.array(pred_shell)
    true_shell = np.array(true_shell)
    shell_pred = abs(pred_shell - true_shell)

    # foot and shell 
    foot=  Image.open('./ML_heatmaps/foot/' + name)
    foot_shell = [foot.getpixel((y,x)) for x,y in nonblack_coords]
    foot_shell = np.array(foot_shell)
    foot_shell = abs(foot_shell - true_shell)

    # pred and foot
    # pred_arr = np.array(pred)
    # nonblack_coords2 = get_non_black(pred_arr)
    # foot_pred = [foot.getpixel((y,x)) for x,y in nonblack_coords2]
    # true_pred = [pred.getpixel((y,x)) for x,y in nonblack_coords2]
    # foot_pred = np.array(foot_pred)
    # true_pred = np.array(true_pred)
    # pred_foot = abs(foot_pred - true_pred)
    pred_foot = abs(foot_shell - pred_shell)
    
    box_plot(foot_shell, pred_foot, shell_pred, name)

In [8]:
for name in os.listdir(os.path.join('./ML_heatmaps/foot/')):
    shell = Image.open('./ML_heatmaps/shell/' + name)
    pred =  Image.open('./pred_to_img/pred/' + name)
    shell_arr = np.array(shell)
    nonblack_coords = get_non_black(shell_arr)
    print(len(nonblack_coords))

    # for x,y in nonblack_coords:
    #     shell.putpixel( (y,x), (255) )
    # pred_arr = np.array(shell, dtype=np.uint8).reshape((128,128))

    # pred_img = Image.fromarray(pred_arr)
    # pred_img.save('./pred_to_img/change4.png')

    pred_shell = [pred.getpixel((y,x)) for x,y in nonblack_coords]
    true_shell = [shell.getpixel((y,x)) for x,y in nonblack_coords]
    pred_shell = np.array(pred_shell)
    true_shell = np.array(true_shell)
    diff = abs(pred_shell - true_shell)
    boxplot(diff, '/aligned/' + name)
    # foot_img = Image.open(os.path.join('./ML_heatmaps/foot/', name))
    # shell_img = Image.open(os.path.join('./ML_heatmaps/shell/', name))

    # coords = get_img_coords(foot_img)
    # pixs = [foot_img.getpixel((coords[i][0], coords[i][1])) for i in range(len(coords))]
    # pred_xyp = np.column_stack((coords, pixs)).tolist()
    # pred_pixs = pipe_ridge.predict(pred_xyp)
    # pred_arr = np.array(pred_pixs, dtype=np.uint8).reshape((128,128))
    # pred_img = Image.fromarray(pred_arr)
    # pred_img.save('./pred_to_img/pred/' + name[:-4] + '.png')

    # img = cv2.imread('./pred_to_img/pred/' + name[:-4] + '.png')
    # img[img == 255] = 0 # change everything to white where pixel is not black
    # cv2.imwrite('./pred_to_img/post_processed/'+name[:-4]+'.png', img)



    # shell_sequence = shell_img.getdata()
    # shell_arr = np.array(shell_sequence)

    # pred = Image.open('./pred_to_img/post_processed/'+name[:-4]+'.png')
    # pred_sequence = pred.getdata()
    # pred_arr = np.array(pred_sequence)

    # diff = abs(shell_arr -  pred_arr)
    # boxplot(diff, name)


2001
2601
1751
1878
2166
1956
1566
1928
1975
2289
2389
2090
2193
1705
1900
2094
1690
2161
2210
1788
2123
1682
1674
1877
1749
1731
2039
1671
2164
1762
2298
1889
1862
2249
2272
2094
2258
1669
1850
2216
1752
2326
1892
2026
1882
2139
1925
2027
1672
2007
2440
2052
1690
1955
2087
2184
1994
2026
2022
2265
1914
1788
1771
2471
2315
1911
2184
1921
1932
2421
2281
2004
1793
1754
1697
2094
2289
2050
2169
2215
1852
1917
1696
1905
1834
2129
1917
2211
1944
2018
1878
1718
1829
1920
2380
2021
1765
2084
1699
1723
1787
2155
2319
1934
1682
1766
2179
2239
2110
1700
2045
1735
1819
1857
1482
1864
1981
1744
1881
2106
1693
1970
1876
2221
1742
2040
1995
1838
1811
1875
2114
2073
1570
2061
2018
1885
2158
1817
1815
1922
2545
2571
1753
2025
1728
1914
1806
1994
1774
2145
2040
1655
1785
1914
2450
2124
1556
2028
2194
2059
1909
2253
1933
2301
2254
1746
2025
1921
1794
2255
2286
2359
2442
1762
1704
2180
1900
1833
1651
2133
2106
2031
2151
2147
1897
1867
2183
1982
1788
2211
1897
2070
1641
2155
1826
1731
1872
1629
1881
2140
